# For a set of csv files, read them in one at a time, perform t-SNE followed by DBSCAN, save figures and results 

In [2]:
import os
import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn import decomposition, metrics
from sklearn.preprocessing import scale, robust_scale
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE
from sklearn.metrics import confusion_matrix

In [3]:
directory = '/Users/tswenson/Documents/Joels/Health_Data_Science/COMPANY_consulting_project/datasets/merged_csvs/'
#directory = '/Users/tswenson/Documents/Joels/Health_Data_Science/COMPANY_consulting_project/datasets/screen_357_cell_plate_1_labeled_merged_csvs/'
df = []
do_boxplots = 0; # binary, 1 indicates that boxplots should be made
#for filename in glob.glob(directory + "LABELLED*.csv"): # reads through all files in this directory looking for *.csv and ignores sub-directories
#for filename in glob.glob(directory + "LABELLED*n09*.csv"): # Just load n09
for filename in glob.glob(directory + "LABELLED*screen_525_cell_plate_1_well_e21*.csv"): # Just load e21
    print("Read in" + filename)
    # Set some variables equal to "None" so that I don't have surprises later
    my_data=None; my_data_headers=None; meta_headers=None; my_data_data_headers=None
    my_scaled_data = None; word_as_num = None;
    tsne_out_mink = None; labels = None
    
    my_data=pd.read_csv(filename, index_col=0)
    my_data_headers = list(my_data)
    meta_headers = ["Width","cell_label","cell_plate","lineage","screen","well","Time"]
    my_data_data_headers = [x for x in my_data_headers if not x in meta_headers]
    # Scale the data columns
    my_scaled_data = scale(my_data[my_data_data_headers])
    # Make a list where cell_label is converted to numbers for plotting
    word_as_num=[]
    print("I CHANGED HEALTHY TO THE SAME COLOR AS UNLABELLED") # This was done because apparently "unlabelled" contained a lot of "healthy" cells
    for word in my_data['cell_label']:
        if word == "unlabelled":
            word_as_num.append("0")
        if word == "blast":
            word_as_num.append("0.5")
        if word == "healthy":
            #word_as_num.append("1")
            word_as_num.append("0")
    ## Assign the last point to have a value of 1 so that the color scale looks the way I want it too
    word_as_num[0] = 1
    lr = 3000 # Set the learning rate for t-SNE
    # Perform t-SNE
    print("Starting t-SNE calculation. Note that for patient samples in which default t-SNE settings do not work, consider modifying the complexity")
    tsne_out_mink = TSNE(metric='euclidean', learning_rate=lr, n_iter=lr, random_state=11).fit_transform(my_scaled_data)
    print("Done with t-SNE calculation")
    # Plot and save the output
    plt.figure(figsize=(20, 10))
    plt.gcf().clear()
    plt.subplot(121)
    plt.scatter(tsne_out_mink[:, 0], tsne_out_mink[:, 1], c= word_as_num, cmap=plt.cm.viridis)
    plt.savefig(filename + "__tsne.png")
    plt.close()
    # Do DBSCAN on the tSNEd data
    print("Starting DBSCAN")
    dbsc = DBSCAN(eps = .6,min_samples=10).fit(tsne_out_mink)
    # eps is sensitive to the number of samples, consider setting programmatically (not tested): eps_ = float(0.6)*float(11000)/float(my_scaled_data.shape[0]) 
    labels = dbsc.labels_
    core_samples = np.zeros_like(labels, dtype = bool)
    core_samples[dbsc.core_sample_indices_] = True
    unique_labels = np.unique(labels)
    print("Plotting DBSCAN")
    # Plot the DBSCAN results
    plt.figure(figsize=(20, 10))
    plt.gcf().clear()
    plt.subplot(121)
    col = np.linspace(0,1,len(unique_labels))
    #col[11] = col[15]; filename = filename + "___CHANGED-COLS__"
    for i in xrange(len(unique_labels)):
        plt.plot(tsne_out_mink[np.where(labels==unique_labels[i])[0], 0], 
             tsne_out_mink[np.where(labels==unique_labels[i])[0], 1],
                #'o', mfc=plt.cm.viridis(col[i]), label=unique_labels[i])
                'o', mfc=plt.cm.nipy_spectral(col[i]), label=unique_labels[i])
                #'o', mfc=plt.cm.prism(col[i]), label=unique_labels[i])
                #'o', mfc=plt.cm.flag(col[i]), label=unique_labels[i])
    plt.legend(loc='best', frameon=False)
    plt.savefig(filename + "__DBSCAN_of_tsne.png")
    plt.close()
    # Summarize the combination of tSNE and DBSCAN results
    tsne_db = [] # This will be a list of lists where the 1st entry is the cluster ID, 2nd: is
    ## how many "unlabelled" are in that cluster, 3rd: number of blast, 4th: number of healthy.
    for i in xrange(len(unique_labels)):
        clst_index = my_data['cell_label'][np.where(labels==unique_labels[i])[0]]
        if clst_index.empty == True:
            print("Cluster index and original data don't line up right. STOP AND FIX")
            break
        tsne_db.append([unique_labels[i],sum(clst_index=='unlabelled') + sum(clst_index=='healthy'),
                         sum(clst_index=='blast')])
    print(tsne_db)
    # Prepare to plot the results as grouped barplots
    print("Plotting grouped barplots")
    tsne_db_df = pd.DataFrame(tsne_db,columns=["Cluster Label","Unlabelled","Blast"])
    tsne_db_df_melted = pd.melt(tsne_db_df,value_vars=["Unlabelled","Blast"],id_vars="Cluster Label")
    plt.gcf().clear()
    ax = sns.barplot(hue="variable",y="value",x="Cluster Label",data=tsne_db_df_melted,log='y')
    plt.savefig(filename + '__barplot.png')
    plt.close()
    if do_boxplots == 1:
        # For each cluster plot boxplots of each feature
        print("Starting boxplots plotting")
        for i in xrange(len(unique_labels)):
            clst_index = my_data.ix[np.where(labels==unique_labels[i])[0],my_data_data_headers]
            clst_index_scaled = my_scaled_data[np.where(labels==unique_labels[i])[0],]
            # Plot the unscaled data
            locations = range(1,(len(my_data_data_headers)+1))
            plt.figure()
            plt.gcf().clear()
            plt.boxplot(clst_index.as_matrix(),positions=locations)
            plt.title("Cluster number " + str(unique_labels[i]))
            plt.ylabel('A.U.')
            plt.xticks(locations, my_data_data_headers,rotation='vertical')
            #plt.yscale('log')
            percs = (clst_index.describe(percentiles=[0.1,0.5,0.9]))
            plt.ylim(percs.iloc[4].min(),percs.iloc[6].max())
            #plt.show()
            plt.savefig(filename + "__boxplots_of_cluster" +str(unique_labels[i])+".png")
            plt.close()
            locations = range(1,(len(my_data_data_headers)+1))
            # Plot the scaled data
            plt.figure()
            plt.gcf().clear()
            plt.boxplot(clst_index_scaled,positions=locations)
            plt.title("Cluster number " + str(unique_labels[i]))
            plt.ylabel('A.U.')
            plt.xticks(locations, my_data_data_headers,rotation='vertical')
            #plt.yscale('log')
            percs_scaled = pd.DataFrame(clst_index_scaled).describe(percentiles=[0.1,0.5,0.9])
            plt.ylim(percs_scaled.iloc[4].min(),percs_scaled.iloc[6].max())
            plt.savefig(filename + "__scaled_boxplots_of_cluster" +str(unique_labels[i])+".png")
            plt.close()
    print("Done with " + filename)

Read in/Users/tswenson/Documents/Joels/Health_Data_Science/COMPANY_consulting_project/datasets/merged_csvs/LABELLED_by_cell_type_screen_525_cell_plate_1_well_e21.csv
I CHANGED HEALTHY TO THE SAME COLOR AS UNLABELLED
Starting t-SNE calculation
Done with t-SNE calculation
Starting DBSCAN
Plotting DBSCAN
[[-1, 210, 2], [0, 23, 0], [1, 2594, 0], [2, 1967, 4], [3, 193, 0], [4, 15, 0], [5, 15, 0], [6, 117, 0], [7, 83, 0], [8, 115, 0], [9, 4, 6], [10, 2121, 0], [11, 47, 160], [12, 106, 0], [13, 11, 0], [14, 14, 1], [15, 20, 0], [16, 207, 0], [17, 10, 0], [18, 87, 0]]
Plotting grouped barplots
Done with /Users/tswenson/Documents/Joels/Health_Data_Science/COMPANY_consulting_project/datasets/merged_csvs/LABELLED_by_cell_type_screen_525_cell_plate_1_well_e21.csv


In [4]:
print(col)
col[10] = col[0]

[ 0.          0.05263158  0.10526316  0.15789474  0.21052632  0.26315789
  0.31578947  0.36842105  0.42105263  0.47368421  0.52631579  0.57894737
  0.63157895  0.68421053  0.73684211  0.78947368  0.84210526  0.89473684
  0.94736842  1.        ]


In [5]:
unique_labels

array([-1,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18])

In [6]:
### Only plot clusters with more than 50 events
tsne_db_df_50 = tsne_db_df[tsne_db_df.sum(axis=1)>110]
print(len(tsne_db_df_50))
tsne_db_df_50_melted = pd.melt(tsne_db_df_50,value_vars=["Unlabelled","Blast"],id_vars="Cluster Label")
ax = sns.barplot(hue="variable",y="value",x="Cluster Label",data=tsne_db_df_50_melted,log='y')
plt.legend(loc = 1,prop={'size':18})
plt.savefig('e21_largeR_clstrs_only__barplot.png')
plt.close()


10


In [7]:
# Add a percentage column
tsne_db_df_50 = tsne_db_df[tsne_db_df.sum(axis=1)>110]
tsne_db_df_50_as_perc = tsne_db_df_50.copy()
tsne_db_df_50_as_perc['Unlabelled_perc'] = 100*tsne_db_df_50_as_perc['Unlabelled']/(tsne_db_df_50_as_perc['Unlabelled']+tsne_db_df_50_as_perc['Blast']) 
tsne_db_df_50_as_perc['Blast_perc'] = 100*tsne_db_df_50_as_perc['Blast']/(tsne_db_df_50_as_perc['Unlabelled']+tsne_db_df_50_as_perc['Blast'])
print(tsne_db_df_50_as_perc)

    Cluster Label  Unlabelled  Blast  Unlabelled_perc  Blast_perc
0              -1         210      2        99.056604    0.943396
2               1        2594      0       100.000000    0.000000
3               2        1967      4        99.797057    0.202943
4               3         193      0       100.000000    0.000000
7               6         117      0       100.000000    0.000000
9               8         115      0       100.000000    0.000000
11             10        2121      0       100.000000    0.000000
12             11          47    160        22.705314   77.294686
13             12         106      0       100.000000    0.000000
17             16         207      0       100.000000    0.000000


In [8]:
tsne_db_df_50_as_perc.shape[0]

10

In [9]:
### Only plot clusters with more than 50 events as a percentage
print(len(tsne_db_df_50))
tsne_db_df_50_as_perc_melted = pd.melt(tsne_db_df_50_as_perc,value_vars=["Unlabelled_perc","Blast_perc"],id_vars="Cluster Label")
ax = sns.barplot(hue="variable",y="value",x="Cluster Label",data=tsne_db_df_50_as_perc_melted)
ax.legend_.remove()
#plt.legend(loc = 1,prop={'size':18})
plt.savefig('e21_largeR_clstrs_only__PERC_barplot.png')
plt.close()


10


In [10]:
tsne_db_df_50

,Cluster Label,Unlabelled,Blast
0,-1,210,2
2,1,2594,0
3,2,1967,4
4,3,193,0
7,6,117,0
9,8,115,0
11,10,2121,0
12,11,47,160
13,12,106,0
17,16,207,0


# Add the following code to the above for loop

In [ ]:
#### I started to work on this, but didn't finish. Psuedo code is mostly done.
'''
#### Export the labelled populations of a well
# Find the fcs file that matches the csv file. 
## First get rid of the .csv, then get rid of the path and the LABELLED,
## now add the path back to the core name identified above, check that 
## it only matches one file, print matches for user to verify
output_name = filename.split(".csv")[0]
filename_fcs = 
for i in xrange(len(unique_labels)):
    fcs_file = FCSParser(path)
    #cluster_data = my_data_keepers.iloc[np.where(labels==unique_labels[i])[0]].astype('float')
    #new_fcs_file = fcs_file.clone(data=cluster_data.as_matrix().astype('float'))
    new_fcs_file = fcs_file.clone(data= fcs_file.data[np.where(labels==unique_labels[i])[0]].astype('float'))
    new_fcs_file.annotation['cluster_label'] = unique_labels[i]
    #new_fcs_file.write_to_file("/Users/tswenson/Documents/Joels/TEST.fcs")
    new_fcs_file.write_to_file(output_name + '__cluster_' + str(unique_labels[i])\
                               + '_exported_'\
                               + datetime.datetime.now().strftime('%Y.%m.%d %I.%M.%S')\
                               + '.fcs')
'''

In [ ]:
'''
# Export Well C3 or my_data
path = '/Users/tswenson/Documents/Joels/Health_Data_Science/COMPANY_consulting_project/datasets/training_set/screen_357_cell_plate_1_well_C3_original.fcs'
output_name = path.split(".fcs")[0]
for i in xrange(len(unique_labels)):
    fcs_file = FCSParser(path)
    #cluster_data = my_data_keepers.iloc[np.where(labels==unique_labels[i])[0]].astype('float')
    #new_fcs_file = fcs_file.clone(data=cluster_data.as_matrix().astype('float'))
    new_fcs_file = fcs_file.clone(data= fcs_file.data[np.where(labels==unique_labels[i])[0]].astype('float'))
    new_fcs_file.annotation['cluster_label'] = unique_labels[i]
    #new_fcs_file.write_to_file("/Users/tswenson/Documents/Joels/TEST.fcs")
    new_fcs_file.write_to_file(output_name + '__cluster_' + str(unique_labels[i])\
                               + '_exported_'\
                               + datetime.datetime.now().strftime('%Y.%m.%d %I.%M.%S')\
                               + '.fcs')
'''